In [215]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")

# Import MySQL Connector/Python 

import mysql.connector as connector

connection =connector.connect(user='admin',password="reason123",db = 'lemon_restaurant')

In [216]:
#create connection to the server
cursor = connection.cursor()

In [217]:
#show the tables in the database
tables = 'SHOW TABLES'
cursor.execute(tables)
for table in cursor:
    print(table)

('booking',)
('customer',)
('delivery',)
('itemorder',)
('menu',)
('ordersview',)
('payment',)
('staff',)


#### Query the database to return Full name, Contact details of customers whose total cost is greater than 130

In [218]:
cost_greater_130 = """SELECT CONCAT(c.firstName,' ',c.lastName) AS "Full Name", c.phoneNumber AS "Phone Number",
                             c.email AS Email,i.totalCost AS "Total Cost"
                    FROM customer c
                    JOIN itemorder i
                    ON c.customerID = i.customerID
                    WHERE totalCost > 130 """

In [219]:
cost_greater_130_df = pd.read_sql(cost_greater_130,con = connection)
cost_greater_130_df

,Full Name,Phone Number,Email,Total Cost
0,Alex Seth,O5588889,alseth@mail.com,300.00
1,Sam Juls,O5577889,samjuls@mail.com,150.00
2,Nita Dan,O6588889,nidan@mail.com,540.55
3,Remi Titus,05924532,rtitus@gmail.com,450.55
4,Ruth Rick,05649867,rrick@mail.com,450.55
5,Ben Glover,067924532,bGlover@gmail.com,480.00
6,Lisa James,079245892,ljames@gmail.com,520.00


#### NB: Use this code if you do not want to read result into a dataframe

#### Retrieve  all menu id with main meal that contributed to costs more than 150

In [220]:
meals_greater_150 = """SELECT m.menuID,m.main AS 'main meal',i.totalCost
                       FROM menu m
                       JOIN itemorder i
                       ON m.menuID = i.menuID 
                       WHERE i.totalCost > 150"""
            

In [221]:
meals_greater_150_df = pd.read_sql(meals_greater_150,con = connection)
meals_greater_150_df

,menuID,main meal,totalCost
0,1,Spaghetti Carbonara,300.00
1,4,Chicken Tikka Masala,540.55
2,5,Chicken Tikka Masala,450.55
3,6,Chicken Tikka Masala,450.55
4,7,Chicken Tikka Masala,480.00
5,8,Jollof,520.00


#### Retrieve customers that have ordered items but haven't booked a table yet with the restaurant. Show the number of Orders for each customer

In [222]:
no_table_booking = """
                  SELECT DISTINCT c.firstName,c.lastName, COALESCE(b.tableNumber,'No table booked') AS 'tableNumber',
                     Count(*) OVER(PARTITION BY c.customerID) AS 'Number of orders'
                  FROM customer c
                  INNER JOIN itemorder i
                  ON c.customerID = i.orderID
                  LEFT JOIN booking b
                  ON b.customerID = c.customerID
                  WHERE b.customerID IS NULL
                  """

In [223]:
no_booking_df = pd.read_sql(no_table_booking, con = connection)
no_booking_df

,firstName,lastName,tableNumber,Number of orders
0,Ruth,Rick,No table booked,1
1,Ben,Glover,No table booked,1
2,Lisa,James,No table booked,1


In [224]:
#close the connection
connection.close()